<a href="https://colab.research.google.com/github/soumilsaha12/MinorProject-ML/blob/main/Model_PL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gspread oauth2client


In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import difflib  # For fuzzy matching

# Step 1: Authenticate and Access Google Sheets
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name('/content/minor-project-439108-cc4a93fe43e4.json', scope)
client = gspread.authorize(creds)

# Step 2: Access Course Data and User Preferences Sheets
course_sheet_id = "1HPsvg6oFH40R_7FY9MS8SYpqPPWNdItbqZFAOCbbNfY"  # ID of the course data sheet
user_prefs_sheet_id = "1_T9r80LVnvwc5PKokC-GSen7GKITdB3JintqkAIKVvs"  # ID of the user preferences sheet

# Open the spreadsheets using the ID
course_sheet = client.open_by_key(course_sheet_id).sheet1  # Open course data sheet
user_prefs_sheet = client.open_by_key(user_prefs_sheet_id).sheet1  # Open user preferences sheet

# Step 3: Get All Course Data and User Preferences
courses = course_sheet.get_all_records()  # Get all course data as a list of dictionaries
user_prefs = user_prefs_sheet.get_all_records()  # Get all user preferences as a list of dictionaries

# Helper function for fuzzy matching topics and calculating similarity score
def get_similarity_score(course_topic, interested_topics):
    max_similarity = 0
    for topic in interested_topics:
        similarity = difflib.SequenceMatcher(None, course_topic.lower(), topic.lower()).ratio()
        max_similarity = max(max_similarity, similarity)  # Track the highest similarity score
    return max_similarity

# Step 4: Process Each User's Preferences and Filter Courses
for user in user_prefs:
    interested_topics = [topic.strip().lower() for topic in user["Interested Fields/Subjects"].split(",")]
    pacing_preference = user["Pacing Preferences"].lower()
    learning_style_preference = user["Preferred Learning Style"].lower()

    print(f"Recommending Courses Based On Your Preferences....")
    print("Here are some courses recommended to you based on your preferences:")
    print(f"Interested Fields: {interested_topics}")
    print(f"Pacing: {pacing_preference}")
    print(f"Learning Style: {learning_style_preference}")
    print("----Recommending----")

    # Step 5: Filter and Rank Courses Based on Similarity Score
    ranked_courses = [
        {
            "course": course,
            "similarity": get_similarity_score(course["Course Topic"], interested_topics)
        }
        for course in courses
        if get_similarity_score(course["Course Topic"], interested_topics) > 0.4  # Only include courses with a similarity score > 0.4
    ]

    # Sort courses by similarity score in descending order
    ranked_courses.sort(key=lambda x: x["similarity"], reverse=True)

    # Step 6: Display Filtered Courses Ranked by Similarity
    if ranked_courses:
        displayed_courses = 0
        for ranked_course in ranked_courses:
            course = ranked_course["course"]
            similarity_score = ranked_course["similarity"]

            # Display courses that match at least one preference even if not exact
            if (pacing_preference in course["Pacing"].lower() or pacing_preference == "any" or similarity_score == 1) and \
               (learning_style_preference in course["Learning Style"].lower() or learning_style_preference == "any" or similarity_score == 1):
                print(f"- {course['Course Name']} ({course['Course Link']}) [Similarity: {similarity_score:.2f}]")
                displayed_courses += 1

        # Fallback: Display more courses if very few are found
        if displayed_courses == 0:
            print("No exact matches, showing courses based on topic relevance:")
            for ranked_course in ranked_courses:
                course = ranked_course["course"]
                similarity_score = ranked_course["similarity"]
                print(f"- {course['Course Name']} ({course['Course Link']}) [Similarity: {similarity_score:.2f}]")
    else:
        print("No courses found matching the preferences.")

    print("\n-------------------------\n")


Recommending Courses Based On Your Preferences....
Here are some courses recommended to you based on your preferences:
Interested Fields: ['artificial intelligence']
Pacing: flexible
Learning Style: visual
----Recommending----
- Machine Learning A-Z (https://www.udemy.com/course/machinelearning/) [Similarity: 1.00]
- Deep Learning A-Z (https://www.udemy.com/course/deeplearning/) [Similarity: 1.00]
- Intro to Artificial Intelligence (https://www.edx.org/course/cs50s-introduction-to-artificial-intelligence-with-python) [Similarity: 1.00]
- Introduction to Machine Learning (https://www.coursera.org/learn/machine-learning/) [Similarity: 1.00]
- Artificial Intelligence Foundations (https://www.udemy.com/course/ai-foundations/) [Similarity: 1.00]
- AI For Everyone (https://www.coursera.org/learn/ai-for-everyone) [Similarity: 1.00]
- Machine Learning with Python (https://www.coursera.org/learn/machine-learning-python) [Similarity: 1.00]
- Computer Vision A-Z (https://www.udemy.com/course/comp